Notebook purpose

- Understand nature of duplicate transactions, explore solutions, document decisions about what duplicates to drop

### Summary

Types of duplicates and how we handle them:

1. `['user_id', 'date', 'amount', 'account_id', 'desc']` are identical* -> drop in main analysis.

2. `['user_id', 'date', 'amount', 'account_id']` are identical and `desc` is similar**, *** -> drop from main analysis.

3. `['user_id', 'date', 'amount', 'account_id']` are identical and `desc` not similar -> keep in main analysis.

4. `['user_id', 'date', 'amount']`, `desc` may or may not differ, but `account_id` differs. This is relevant if there are (many) duplicated accounts, in which case a different account number is no guarantee for a different account. -> ignore for now, ask MDB to share list of duplicated accounts.

\* This includes pairs where description for both txns is `<mdbremoved>`, in which case we assume that the same description is being masked.

\** "similar" is defined below.

\*** In this category we include cases where desc of one txn is `<mdbremoved>` while other isn't, so even though descriptions are not similar, we assume that what is being masked by `<mdbremoved>` in one description is similar to what is visible in the other.

Solution steps:

- preprocess using cleaning func to eliminate extraneous chars
- drop type 1 dups
- of remaining dups, drop those for which all WORDS of short desc are contained in long desc

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws
import entropy.data.cleaners as cl

sns.set_style('whitegrid')
pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_parquet('~/tmp/entropy_X77.parquet')

Preprocessed df

In [3]:
def clean_desc(df):
    """Removes extraneous characters that often create duplicates."""
    df['desc'] = (df.desc
                  .str.replace(r'x{2,}', '', regex=True)    # used to mask card numbers
                  .str.replace(r'\s{2,}', ' ', regex=True)  # multiple spaces
                  .str.replace(' - vis', '', regex=False)   # visa debit card suffix
                  .str.replace(' - dd', '', regex=False)    # debit direct suffix
                  .str.replace(' - d/d', '', regex=False)    # debit direct suffix
                  .str.replace(' - s/o', '', regex=False)   # standing order suffix
                  .str.replace('- pos', '', regex=False)    # point of sale suffix
                  .str.replace(' - )))', '', regex=False)   # crypic suffix
                  .str.replace(' - ', ' ', regex=False))    # hypen
    return df

df = clean_desc(df)

Helper functions

In [3]:
def distr(x):
    pcts = [.01, .05, .1, .25, .50, .75, .90, .95, .99]
    return x.describe(percentiles=pcts).round(2)

def duplicates_sample(df, col_subset, n=100, seed=2312):
    """Draws sample of size n of duplicate txns as defined by col_subset."""
    dups = df[df.duplicated(subset=col_subset, keep=False)].copy()
    dups['group'] = dups.groupby(col_subset).ngroup()
    unique_groups = np.unique(dups.group)
    rng = np.random.default_rng(seed=seed)
    sample = rng.choice(unique_groups, size=n)
    return dups[dups.group.isin(sample)]

## Case studies

Below three case studies of duplicates

In [ ]:
dh.user_date_data(df, 35177, '1 Jan 2020')

In [ ]:
dh.user_date_data(df, 362977, '1 Jan 2020')

In [ ]:
dh.user_date_data(df, 467877, '1 Jan 2020')

## Type 1 duplicates

### Definition
- `['user_id', 'date', 'amount', 'account_id', 'desc']` are identical.
 
- This includes transactions where desc for both is `<mdbremoved>`, where we have to make a call whether or not to assume they are the same.

- Reasons for false positives (FP): user makes two identical transactions on the same day (or on subsequent days for txns that appear with a delay). Plausible cases are coffee and betting shop txns. However, inspection suggests that the vast majority of cases are duplicates, as they are txns that are unlikely to result from multiple purchases on the same day.


### Decision

- We delete dups for main analysis and do robustness check withouth deleting them
- We tread cases where both descriptions are `<mdbremoved>` no different from others, even though it's somewhat more likely that they are genuinely different transactions

In [4]:
col_subset = ['user_id', 'date', 'amount', 'account_id', 'desc']
dup_var = 'dup1'

df[dup_var] = df.duplicated(subset=col_subset)

### Prevalence and value

How prevalent are duplicates?

In [5]:
n_df = len(df)
n_dups = len(df[df[dup_var]])
n_users_dups = df[df[dup_var]].user_id.nunique()
n_users_df = df.user_id.nunique()
txt = 'About {:.1%} of transactions across {:.0%} of users are potential dups.'
print(txt.format(n_dups / n_df, n_users_dups / n_users_df))

About 1.7% of transactions across 97% of users are potential dups.


Gross value of duplicated txns

In [6]:
gross_value = df[df[dup_var]].set_index('user_id').amount.abs().groupby('user_id').sum()
distr(gross_value)

count       417.00
mean       4622.48
std       15167.33
min           0.17
1%            4.07
5%           19.30
10%          61.06
25%         236.30
50%         836.96
75%        2736.70
90%        9293.31
95%       17393.81
99%       59012.20
max      183754.34
Name: amount, dtype: float64

Most frequent txns description

In [7]:
df[df[dup_var]].desc.value_counts(dropna=False)[:10]

<mdbremoved>                         2050
<mdbremoved>                          517
<mdbremoved> ft                       357
b365 moto                             263
tfl travel charge tfl.gov.uk/cp       167
www.skybet.com cd 9317                165
<mdbremoved> - s/o                    158
<mdbremoved> so                       156
bank giro credit ref <mdbremoved>     147
betfair.-purchase                     146
Name: desc, dtype: int64

Most frequent auto tag

In [8]:
df[df[dup_var]].tag_auto.value_counts(dropna=False)[:10]

NaN                           6240
transfers                     2961
gambling                      2185
enjoyment                     1535
public transport              1076
lunch or snacks               1048
bank charges                   823
entertainment, tv, media       562
cash                           516
food, groceries, household     506
Name: tag_auto, dtype: int64

Proportion of txns per auto tag that are duplicated

In [9]:
txns_per_tag_overall = df.tag_auto.value_counts(dropna=False)
txns_per_tag_duplicated = df[df[dup_var]].tag_auto.value_counts(dropna=False) 
p_dup_per_tag = (txns_per_tag_duplicated / txns_per_tag_overall)
p_dup_per_tag.sort_values(ascending=False)[:40]

investment - other               0.214953
gambling                         0.159303
mobile app                       0.151954
isa                              0.088095
tradesmen fees                   0.076923
flights                          0.049423
parking                          0.046014
payment protection insurance     0.044776
bills                            0.040082
home appliance insurance         0.039448
games and gaming                 0.038053
supermarket                      0.035669
road charges                     0.032078
pension or investments           0.030481
pet insurance                    0.027899
bank charges                     0.026923
refunded purchase                0.026788
public transport                 0.026075
child - everyday or childcare    0.024896
fines                            0.024390
NaN                              0.024055
gym membership                   0.023636
postage / shipping               0.023256
entertainment, tv, media         0

### Inspect dups

In [10]:
duplicates_sample(df, col_subset, n=3, seed=None)

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,savings,dup1,group
60192,806078787,2018-08-12,35177,-167.00,<mdbremoved>,NaN,None,None,False,xxxx 0,2014-02-14,20k to 30k,1990.0,2017-05-25,724235,2020-08-14 20:59:00,hsbc,current,2020-08-12,1900-01-01,False,844.299988,NaN,NaN,NaN,NaN,c,201808,-199.038452,55600.742188,False,False,522
60193,806078788,2018-08-12,35177,-167.00,<mdbremoved>,NaN,None,None,False,xxxx 0,2014-02-14,20k to 30k,1990.0,2017-05-25,724235,2020-08-14 20:59:00,hsbc,current,2020-08-12,1900-01-01,False,844.299988,NaN,NaN,NaN,NaN,c,201808,-199.038452,55600.742188,False,True,522
62590,626454562,2019-09-09,35177,530.00,<mdbremoved> bp,NaN,None,None,False,xxxx 0,2014-02-14,20k to 30k,1990.0,2017-05-25,724235,2020-08-14 20:59:00,hsbc,current,2019-10-14,1900-01-01,True,844.299988,non merchant mbl,NaN,rent,rent,c,201909,1096.661011,65721.826705,False,False,623
62591,626454563,2019-09-09,35177,530.00,<mdbremoved> bp,NaN,None,None,False,xxxx 0,2014-02-14,20k to 30k,1990.0,2017-05-25,724235,2020-08-14 20:59:00,hsbc,current,2019-10-14,1900-01-01,True,844.299988,non merchant mbl,NaN,rent,rent,c,201909,1096.661011,65721.826705,False,True,623
630678,659371722,2019-11-19,379477,7.35,contactless payment caffe paradiso,NaN,spend,services,True,n13 4,2017-04-30,20k to 30k,1980.0,2018-03-17,913197,2020-08-16 16:54:00,nationwide,current,2019-12-10,1900-01-01,True,4203.459961,NaN,lunch or snacks,NaN,lunch or snacks,c,201911,18624.255859,43956.789062,False,False,8401
630685,791975455,2019-11-19,379477,7.35,contactless payment caffe paradiso,NaN,spend,services,True,nw10 3,2017-04-30,20k to 30k,1980.0,2018-03-17,913197,2020-08-16 16:54:00,nationwide,current,2020-06-24,1900-01-01,True,4203.459961,NaN,lunch or snacks,NaN,lunch or snacks,c,201911,18624.255859,43956.789062,False,True,8401


In [12]:
df = df.drop_duplicates(subset=col_subset)

## Type 2 dups

### Definition

- `['user_id', 'date', 'amount', 'account_id']` are identical, `desc` is different but similar, or one `desc` contains `<mdbremoved>` and the other one isn't.

### Decision

- Inspection suggests that in most cases, similar but different desc strings result from slight editing of the string, e.g. by removing unnecessary punctuation characters or (as discussed in MDB documentation) by revealing additional information that a new algorighm has classified as non-sensitive.

In [13]:
col_subset = ['user_id', 'date', 'amount', 'account_id']
dup_var = 'dup2'

df[dup_var] = df.duplicated(subset=col_subset)

### Prevalence and value

How prevalent are duplicates?

In [14]:
n_df = len(df)
n_dups = len(df[df[dup_var]])
n_users_dups = df[df[dup_var]].user_id.nunique()
n_users_df = df.user_id.nunique()
txt = 'About {:.1%} of transactions across {:.0%} of users are potential dups.'
print(txt.format(n_dups / n_df, n_users_dups / n_users_df))

About 1.9% of transactions across 99% of users are potential dups.


Gross value of duplicated txns

In [15]:
gross_value = df[df[dup_var]].set_index('user_id').amount.abs().groupby('user_id').sum()
distr(gross_value)

count       426.00
mean       2543.32
std        8323.64
min           2.50
1%            7.72
5%           41.88
10%          91.53
25%         295.48
50%         881.08
75%        2102.15
90%        4924.51
95%        6928.04
99%       25746.48
max      106598.39
Name: amount, dtype: float64

Most frequent txns description

In [16]:
df[df[dup_var]].desc.str[:12].value_counts(dropna=False)[:10]

<mdbremoved>    3460
daily od fee    1894
int'l xxxxxx     944
card payment     490
direct debit     439
contactless      410
visa purchas     346
tfl travel c     343
tfl.gov.uk/c     288
call ref.no.     273
Name: desc, dtype: int64

### Similarity score

Most frequent auto tag

In [64]:
import difflib
import functools
import collections

from fuzzywuzzy import fuzz

DescAndId = collections.namedtuple('DescAndID', ['desc', 'id'])
shortest_first = functools.partial(sorted, key=lambda x: len(x.desc))

def similarity_score(group):
    """Return similarity score between longest string in group and all others."""
    cols = list(group.columns)
    group['score_difflib'] = np.nan
    group['score_fuzz'] = np.nan
    items = [DescAndId(*item) for item in zip(group.desc, group.id)]
    shortest, *others = shortest_first(items)
    for o in others:
        group.loc[group.id == o.id, 'score_difflib'] = difflib.SequenceMatcher(None, shortest.desc, o.desc).ratio()
        group.loc[group.id == o.id, 'score_fuzz'] = fuzz.partial_ratio(shortest.desc, o.desc)
    return group[['score_difflib', 'score_fuzz'] + cols]

dd = duplicates_sample(df, col_subset, n=3, seed=None).groupby('group').apply(similarity_score)
dd

,score_difflib,score_fuzz,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,savings,dup1,dup2,group
658859,0.987654,98.0,267789798,2017-05-02,396277,671.00,<mdbremoved> type: standing order - s/o,NaN,transfers,tsransfer,False,ca22 2,2017-09-24,NaN,1981.0,2017-09-24,792705,2017-09-24 15:54:00,natwest bank,other,2017-09-25,2017-11-13,True,8630.809570,non merchant mbl,transfers,NaN,NaN,u,201705,18569.878906,0.540000,False,False,False,14041
658860,NaN,NaN,267789796,2017-05-02,396277,671.00,<mdbremoved> type: standing order - s/o,NaN,transfers,tsransfer,False,ca22 2,2017-09-24,NaN,1981.0,2017-09-24,792705,2017-09-24 15:54:00,natwest bank,other,2017-09-25,2017-11-13,True,8630.809570,non merchant mbl,transfers,NaN,NaN,u,201705,18569.878906,0.540000,False,False,True,14041
690643,NaN,NaN,478359011,2018-11-23,411677,8.00,02nov a/c xxxxxx92 - chg,NaN,spend,finance,True,se10 8,2018-01-02,NaN,1984.0,2018-01-02,847165,2020-03-11 16:04:00,natwest bank,current,2018-11-29,2019-04-09,True,-614.969971,account provider,bank charges,NaN,NaN,u,201811,-704.140137,59531.070312,False,False,False,14409
690644,0.137931,29.0,478359009,2018-11-23,411677,8.00,unarrnged od usage 02 <mdbremoved>,NaN,spend,finance,True,se10 8,2018-01-02,NaN,1984.0,2018-01-02,847165,2020-03-11 16:04:00,natwest bank,current,2018-11-29,1900-01-01,True,-614.969971,account provider,bank charges,NaN,bank charges,c,201811,-704.140137,59531.070312,False,False,True,14409
1294130,NaN,NaN,771163718,2019-10-21,583677,3.69,daily od fee 19/10,NaN,spend,finance,True,pe23 5,2020-05-12,20k to 30k,1987.0,2020-05-12,1688628,2020-08-09 22:03:00,lloyds bank,current,2020-05-13,1900-01-01,True,-2173.780029,account provider,bank charges,NaN,bank charges,c,201910,-1936.300171,46319.640625,False,False,False,22383
1294131,0.888889,89.0,771167992,2019-10-21,583677,3.69,daily od fee 20/10,NaN,spend,finance,True,pe23 5,2020-05-12,20k to 30k,1987.0,2020-05-12,1688628,2020-08-09 22:03:00,lloyds bank,current,2020-05-13,1900-01-01,True,-2173.780029,account provider,bank charges,NaN,bank charges,c,201910,-1936.300171,46319.640625,False,False,True,22383


In [73]:
def clean_desc(df):
    """Removes extraneous characters that often create duplicates."""
    df = df.copy()
    df['desc'] = (df.desc
#                   .str.replace(' - vis', '', regex=False)   # visa debit card suffix
#                   .str.replace(' - dd', '', regex=False)    # debit direct suffix
#                   .str.replace(' - d/d', '', regex=False)    # debit direct suffix
#                   .str.replace(' - s/o', '', regex=False)   # standing order suffix
#                   .str.replace('- pos', '', regex=False)    # point of sale suffix
#                   .str.replace(' - dpc', '', regex=False)   # digital banking suffix
#                   .str.replace(' - gbr', '', regex=False)   # gbr suffix
                  .str.replace(r' - .{2, 3}', '', regex=True)
                  .str.replace(' - )))', '', regex=False)   # crypic suffix
                  .str.replace(' - ', ' ', regex=False)    # hypen
                  .str.replace('-', ' ', regex=False)     # hypen
                  .str.replace(r'\*{2,}', '', regex=True)    # used to mask card numbers
                  .str.replace(r'x{2,}', '', regex=True)    # used to mask card numbers
                  .str.replace(r'\s{2,}', ' ', regex=True)  # multiple spaces
                 )
    return df

clean_desc(dd).desc

658859     <mdbremoved> type: standing order s/o
658860     <mdbremoved> type: standing order s/o
690643                          02nov a/c 92 chg
690644        unarrnged od usage 02 <mdbremoved>
1294130                       daily od fee 19/10
1294131                       daily od fee 20/10
Name: desc, dtype: object

In [209]:
df = d

In [128]:
print('- )))')

- )))


Non-dup patterns
- daily od fee dd/mm
- long string with DDmonYY date that differs

Dup patterns (not captured by score_fuzz)
- split \d+\w+ into \d+ \w+
- all WORDS of short appear in long (as WORD or part of WORD) (533726, 533730; 58878, 58893)  but not (510409, 510410)
- Additional info added (e.g. card digit revealed, more card info added)
- Additional info revealed and words rearranged
- Both contain mask and other stuff

### Inspect dups

In [50]:
dup_txn_sample(df, dup_subset, n=2, seed=None)

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,savings,dup1,dup2,group
74671,527461874,2019-03-22,35177,20.0,cash notemac mar22 - one click se@xx:xx - atm,NaN,spend,other_spend,False,xxxx 0,2014-02-14,20k to 30k,1990.0,2017-05-25,724235,2020-08-14 20:59:00,hsbc,current,2019-03-23,2019-05-06,True,844.299988,personal,cash,NaN,cash,u,201903,200.301453,79785.007812,False,False,False,1756
74673,806078460,2019-03-22,35177,20.0,<mdbremoved> atm,NaN,spend,other_spend,False,xxxx 0,2014-02-14,20k to 30k,1990.0,2017-05-25,724235,2020-08-14 20:59:00,hsbc,current,2020-08-12,1900-01-01,True,844.299988,personal,cash,NaN,cash,c,201903,200.301453,79785.007812,False,False,True,1756
607714,191201530,2017-01-30,373777,50.0,"cash withdrawal at note machine atm morley service, leeds,50.00 gbp , on xx-xx-2017",NaN,spend,other_spend,False,ls27 8,2017-03-07,NaN,1963.0,2017-03-07,680911,2017-06-28 22:55:00,santander,current,2017-03-08,2017-10-23,True,NaN,personal,cash,NaN,cash,u,201701,NaN,29253.576563,False,False,False,12750
607716,191201527,2017-01-30,373777,50.0,"direct debit payment to h3g ref xxxxxxxxxxxxxx0117, mandate no 0013",hutchison 3g,spend,communication,False,ls27 8,2017-03-07,NaN,1963.0,2017-03-07,680911,2017-06-28 22:55:00,santander,current,2017-03-08,1900-01-01,True,NaN,hutchison 3g,mobile,NaN,mobile,c,201701,NaN,29253.576563,False,False,True,12750
607718,191201533,2017-01-30,373777,50.0,"direct debit payment to h3g ref xxxxxxxxxxxxxx0117, mandate no 0003",hutchison 3g,spend,communication,False,ls27 8,2017-03-07,NaN,1963.0,2017-03-07,680911,2017-06-28 22:55:00,santander,current,2017-03-08,1900-01-01,True,NaN,hutchison 3g,mobile,NaN,mobile,c,201701,NaN,29253.576563,False,False,True,12750


,score_difflib,score_fuzz,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,savings,dup1,dup2,group
918495,0.653061,67.0,558421618,2019-05-29,490277,3.0,waitrose 732 cd 6728 deb,waitrose,spend,household,True,n16 6,2019-02-11,NaN,1992.0,2019-02-11,1188710,2020-03-09 21:08:00,lloyds bank,current,2019-05-30,1900-01-01,True,473.660004,waitrose,"food, groceries, household",NaN,"food, groceries, household",c,201905,962.689209,17570.730469,False,False,False,17125
918496,NaN,NaN,558421619,2019-05-29,490277,3.0,tfl travel ch cd 6728 deb,tfl,spend,travel,True,n16 6,2019-02-11,NaN,1992.0,2019-02-11,1188710,2020-03-09 21:08:00,lloyds bank,current,2019-05-30,1900-01-01,True,473.660004,tfl,public transport,NaN,public transport,c,201905,962.689209,17570.730469,False,False,True,17125


- lowest same: .82 most, 72
- highest different: .71

Cases for decision:
- One mdbremoved, the other unremoved has very low score -> treat as dup or not? (could use 100 fuzz score to exclude)
- daily od fee _date_ only differs in date and has very high score but is different - exclude manually or handle automatically. some other descs that contain differing dates. could match dates and ensure they differ
- remove `-`, `)` and `(`

In [305]:
dd.desc.str.strip

<bound method StringMethods.normalize of <pandas.core.strings.accessor.StringMethods object at 0x17c3c1490>>

In [51]:
dup_subset = ['user_id', 'date', 'amount', 'account_id']
dup_var = 'dup2'
df[dup_var] = df.duplicated(subset=dup_subset)

In [101]:
from fuzzywuzzy import fuzz
from functools import partial

longest_first = partial(sorted, key=lambda x: len(x), reverse=True)

for idx, data in dd.groupby('group'):
    longest, *others = longest_first(data.desc.values)
    print(longest)
    for other in others:
        print('   {}'.format(other))
        print('   {}'.format(fuzz.partial_ratio(longest, other)), end='\n\n')


lnk sk store, 44/4 cd 8050 12jul15
   sby tamworth cd 8050 13jul15
   68

lnk sk store, 44/4 cd 8050 13dec15
   lnk star news coto cd 9447 12dec15
   59

the boathouse bras cd 4720 deb
   tesco stores 6711 cd 4720 deb
   62

<mdbremoved>
   <mdbremoved>
   100

   <mdbremoved>
   100

<mdbremoved> xxxxxx xxxx5560
   <mdbremoved>
   100

bank credit <mdbremoved>
   bank credit <mdbremoved>
   100

xxxxxx xxxx0290 internet transfer
   xxxxxx xxxx8658 internet transfer
   88

32 red cd 7512 deb
   32 red cd 7512 deb
   100

non-stg purch fee cd 6710 deb
   non-stg purch fee cd 6710 deb
   100

   non-stg purch fee cd 6710 deb
   100

   non-stg purch fee cd 6710 deb
   100

card payment to iz *canopy market,2.00 gbp, rate 1.00/gbp on 10-07-2020
   card payment to iz *crosstown,2.00 gbp, rate 1.00/gbp on 10-07-2020
   85



## Type 1 dups

In [2]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws
import entropy.data.cleaners as cl
import entropy.data.helpers as dh

sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
